In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp


In [4]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset


In [5]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


In [6]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


In [7]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [8]:
create_model_inputs()

{'fixed acidity': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fixed acidity')>,
 'volatile acidity': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'volatile acidity')>,
 'citric acid': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'citric acid')>,
 'residual sugar': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'residual sugar')>,
 'chlorides': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'chlorides')>,
 'free sulfur dioxide': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'free sulfur dioxide')>,
 'total sulfur dioxide': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'total sulfur dioxide')>,
 'density': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'density')>,
 'pH': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'pH')>,
 'sulphates': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'sulphates')>,
 'alcohol': <KerasTensor: shap

In [9]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\lomp\tensorflow_datasets\wine_quality\white\1.0.0.incompleteHJS9ZT\wine_quality-train.tfrec…

Dataset wine_quality downloaded and prepared to C:\Users\lomp\tensorflow_datasets\wine_quality\white\1.0.0. Subsequent calls will reuse this data.


In [10]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)


Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 21ms/step - loss: 31.4088 - root_mean_squared_error: 5.6044 - val_loss: 32.3489 - val_root_mean_squared_error: 5.6876
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 29.1470 - root_mean_squared_error: 5.3988 - val_loss: 30.0166 - val_root_mean_squared_error: 5.4787
Epoch 3/100
17/17 [==============================] - 0s 3ms/step - loss: 27.2400 - root_mean_squared_error: 5.2192 - val_loss: 27.6756 - val_root_mean_squared_error: 5.2608
Epoch 4/100
17/17 [==============================] - 0s 3ms/step - loss: 25.4189 - root_mean_squared_error: 5.0417 - val_loss: 25.4382 - val_root_mean_squared_error: 5.0436
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: 23.6594 - root_mean_squared_error: 4.8641 - val_loss: 23.3411 - val_root_mean_squared_error: 4.8313
Epoch 6/100
17/17 [==============================] - 0s 3ms/step - loss: 21.9578 - root_mean_squared_error

17/17 [==============================] - 0s 4ms/step - loss: 0.6820 - root_mean_squared_error: 0.8258 - val_loss: 0.6889 - val_root_mean_squared_error: 0.8300
Epoch 49/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6768 - root_mean_squared_error: 0.8227 - val_loss: 0.6803 - val_root_mean_squared_error: 0.8248
Epoch 50/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6727 - root_mean_squared_error: 0.8202 - val_loss: 0.6745 - val_root_mean_squared_error: 0.8213
Epoch 51/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6662 - root_mean_squared_error: 0.8162 - val_loss: 0.6692 - val_root_mean_squared_error: 0.8181
Epoch 52/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6622 - root_mean_squared_error: 0.8138 - val_loss: 0.6639 - val_root_mean_squared_error: 0.8148
Epoch 53/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6559 - root_mean_squared_error: 0.8099 - val_loss: 0.6591 - val_root_mean_squ

Epoch 96/100
17/17 [==============================] - 0s 3ms/step - loss: 0.5663 - root_mean_squared_error: 0.7526 - val_loss: 0.5631 - val_root_mean_squared_error: 0.7504
Epoch 97/100
17/17 [==============================] - 0s 3ms/step - loss: 0.5681 - root_mean_squared_error: 0.7537 - val_loss: 0.5613 - val_root_mean_squared_error: 0.7492
Epoch 98/100
17/17 [==============================] - 0s 3ms/step - loss: 0.5653 - root_mean_squared_error: 0.7519 - val_loss: 0.5608 - val_root_mean_squared_error: 0.7489
Epoch 99/100
17/17 [==============================] - 0s 3ms/step - loss: 0.5644 - root_mean_squared_error: 0.7513 - val_loss: 0.5603 - val_root_mean_squared_error: 0.7485
Epoch 100/100
17/17 [==============================] - 0s 3ms/step - loss: 0.5677 - root_mean_squared_error: 0.7534 - val_loss: 0.5598 - val_root_mean_squared_error: 0.7482
Model training finished.
Train RMSE: 0.751
Evaluating model performance...
Test RMSE: 0.748


In [11]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")


Predicted: 5.3 - Actual: 5.0
Predicted: 6.2 - Actual: 6.0
Predicted: 5.6 - Actual: 5.0
Predicted: 5.9 - Actual: 5.0
Predicted: 6.1 - Actual: 7.0
Predicted: 6.5 - Actual: 8.0
Predicted: 6.3 - Actual: 6.0
Predicted: 6.1 - Actual: 7.0
Predicted: 5.3 - Actual: 6.0
Predicted: 6.3 - Actual: 6.0
